In [12]:
import numpy as np
import matplotlib.pyplot as plt
# from qiskit_nature.drivers import Molecule
# from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver,ElectronicStructureDriverType
# from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
# from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
# from qiskit_nature.converters.second_quantization import QubitConverter
# from qiskit_nature.mappers.second_quantization import ParityMapper,JordanWignerMapper,BravyiKitaevMapper
from scipy.optimize import minimize
# from qiskit.algorithms.optimizers import COBYLA, SPSA,SLSQP,GradientDescent
# from qiskit.opflow import TwoQubitReduction,Z2Symmetries
# from qiskit import BasicAer, Aer
# from qiskit.utils import QuantumInstance
from qiskit.quantum_info import Pauli
import time

In [13]:
import time
import numpy as np
from qiskit_aer import Aer
from scipy.optimize import minimize
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile

In [17]:
def get_qubit_op(h_pq, h_ijkl):
    
    h1_a = np.array(h_pq, dtype=float)    
    h2_aa = np.array(h_ijkl, dtype=float)

    hamiltonian = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa)

    mapper = JordanWignerMapper()
    fermionic_op = hamiltonian.second_q_op()
    qubit_op = mapper.map(fermionic_op)

    return qubit_op

In [18]:
h_pq = [[4.9348022,          0],
        [        0, 19.7392088]]

size = len(h_pq)
h_ijkl = np.zeros((size, size, size, size))

qubit_op = get_qubit_op(h_pq, h_ijkl)

In [21]:
qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII'],
              coeffs=[24.674011 +0.j, -2.4674011+0.j, -9.8696044+0.j, -2.4674011+0.j,
 -9.8696044+0.j])

In [19]:
num_particles = 1

In [85]:
def define_ansatz(theta):
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    ansatz = QuantumCircuit(q,c)
    pi = np.pi
    ansatz.rx(pi,q[1])
    ansatz.rx(-pi/2,q[1])
    ansatz.ry(pi/2,q[0])
    ansatz.cx(q[0],q[1])
    ansatz.rz(theta[0],q[1])
    ansatz.cx(q[0],q[1])
    ansatz.ry(-pi/2,q[0])
    ansatz.rx(pi/2,q[1])
    return (ansatz)

In [87]:
def VQE_circuit(theta, n, depth): 
    """
    Creates a variational-form RY ansatz.
    
    theta: (depth+1 x n) matrix of rotation angles.
    n    : number of qbits.
    depth: number of layers.
    """
        
    if len(theta.ravel()) != ((depth+1) * n):        
        raise ValueError("Theta cannot be reshaped as a (depth+1 x n) matrix")

    theta.shape = (depth + 1, n)

    # Define the Quantum and Classical Registers
    q = QuantumRegister(n)
    c = ClassicalRegister(n)

    # Build the circuit for the ansatz
    circuit = QuantumCircuit(q, c)

    # Put all the qbits in the |+> state
    for i in range(n):
        circuit.ry(theta[0,i],q[i])
    circuit.barrier()
    
    # Now introduce the z-gates and RY-gates 'depth' times
    for j in range(depth):
        # Apply controlled-z gates
        for i in range(n-1):
            circuit.cz(q[i], q[i+1])

        # Introduce RY-gates
        for i in range(n):
            circuit.ry(theta[j+1,i],q[i])
        circuit.barrier()
    
    # Close the circuit with qbits measurements
    circuit.measure(q, c)
    
    return circuit

In [92]:
pi = np.pi

theta = np.array([[1, 0, 1, 2, 5],
                  [1, 0, 1, 2, 5],
                  [1, 0, 1, 2, 5],
                  [1, 0, 1, 2, 5],
                  [1, 0, 1, 2, 5]])

VQE_circuit(theta, n=5, depth=4).draw()

┌───────┐ ░    ┌───────┐                            ░    ┌───────┐»
q218_0: ┤ Ry(1) ├─░──■─┤ Ry(1) ├────────────────────────────░──■─┤ Ry(1) ├»
        ├───────┤ ░  │ └───────┘┌───────┐                   ░  │ └───────┘»
q218_1: ┤ Ry(0) ├─░──■─────■────┤ Ry(0) ├───────────────────░──■─────■────»
        ├───────┤ ░        │    └───────┘┌───────┐          ░        │    »
q218_2: ┤ Ry(1) ├─░────────■────────■────┤ Ry(1) ├──────────░────────■────»
        ├───────┤ ░                 │    └───────┘┌───────┐ ░             »
q218_3: ┤ Ry(2) ├─░─────────────────■────────■────┤ Ry(2) ├─░─────────────»
        ├───────┤ ░                          │    ├───────┤ ░             »
q218_4: ┤ Ry(5) ├─░──────────────────────────■────┤ Ry(5) ├─░─────────────»
        └───────┘ ░                               └───────┘ ░             »
 c77: 5/══════════════════════════════════════════════════════════════════»
                                                                          »
«                                    ░    ┌───────┐                           »
«q218_0: ────────────────────────────░──■─┤ Ry(1) ├───────────────────────────»
«        ┌───────┐                   ░  │ └───────┘┌───────┐                  »
«q218_1: ┤ Ry(0) ├───────────────────░──■─────■────┤ Ry(0) ├──────────────────»
«        └───────┘┌───────┐          ░        │    └───────┘┌───────┐         »
«q218_2: ────■────┤ Ry(1) ├──────────░────────■────────■────┤ Ry(1) ├─────────»
«            │    └───────┘┌───────┐ ░                 │    └───────┘┌───────┐»
«q218_3: ────■────────■────┤ Ry(2) ├─░─────────────────■────────■────┤ Ry(2) ├»
«                     │    ├───────┤ ░                          │    ├───────┤»
«q218_4: ─────────────■────┤ Ry(5) ├─░──────────────────────────■────┤ Ry(5) ├»
«                          └───────┘ ░                               └───────┘»
« c77: 5/═════════════════════════════════════════════════════════════════════»
«                                                                             »
«         ░    ┌───────┐                            ░ ┌─┐            
«q218_0: ─░──■─┤ Ry(1) ├────────────────────────────░─┤M├────────────
«         ░  │ └───────┘┌───────┐                   ░ └╥┘┌─┐         
«q218_1: ─░──■─────■────┤ Ry(0) ├───────────────────░──╫─┤M├─────────
«         ░        │    └───────┘┌───────┐          ░  ║ └╥┘┌─┐      
«q218_2: ─░────────■────────■────┤ Ry(1) ├──────────░──╫──╫─┤M├──────
«         ░                 │    └───────┘┌───────┐ ░  ║  ║ └╥┘┌─┐   
«q218_3: ─░─────────────────■────────■────┤ Ry(2) ├─░──╫──╫──╫─┤M├───
«         ░                          │    ├───────┤ ░  ║  ║  ║ └╥┘┌─┐
«q218_4: ─░──────────────────────────■────┤ Ry(5) ├─░──╫──╫──╫──╫─┤M├
«         ░                               └───────┘ ░  ║  ║  ║  ║ └╥┘
« c77: 5/══════════════════════════════════════════════╩══╩══╩══╩══╩═
«                                                      0  1  2  3  4

In [104]:
def cost_function_C(results, weights):
    
    # the eigenstates obtained by the evaluation of the circuit
    eigenstates = list(results.keys())
    
    # how many times each eigenstate has been sampled
    abundancies = list(results.values())
    
    # number of shots 
    shots = sum(results.values())
    
    # initialize the cost function
    cost = 0
    
    for k in range(len(eigenstates)):
        # ndarray of the digits extracted from the eigenstate string 
        x = np.array([int(num) for num in eigenstates[k]])
        # Cost function due to the k-th eigenstate
        cost = cost + x.dot(weights.dot(1-x)) * abundancies[k]
    
    return -cost / shots

In [131]:
def Energy_general(theta, R, num_qubits, gates_list_hamiltonian, num_elements_hamiltonian):

    shots = 2**14

    provider = QiskitRuntimeService()          
    backend = provider.backend('ibmq_qasm_simulator')

    E_sim = []
    Energy_meas = []
    index_param = 0

    for measurement in gates_list_hamiltonian[1:]:

        index_param += 1 
        n = num_qubits 
        q = QuantumRegister(num_qubits)
        c = ClassicalRegister(num_qubits)
        hamiltonian_circuit = QuantumCircuit(q,c)


        
        for sing_gate in measurement:
            n -= 1
            if sing_gate == Pauli('Z'):
                hamiltonian_circuit.measure(n,n)
            if sing_gate == Pauli('X'):
                hamiltonian_circuit.h(q[n])
            hamiltonian_circuit.measure(n,n)
            if sing_gate == Pauli('Y'):
                hamiltonian_circuit.sdg(q[n])
                hamiltonian_circuit.h(q[n])
                hamiltonian_circuit.measure(n,n)

                print(hamiltonian_circuit.draw())

        qc = VQE_circuit(theta, n=num_qubits, depth=num_qubits-1).compose(hamiltonian_circuit)

        qc_trans = transpile(qc, backend)
        counts = backend.run(qc_trans,shots=shots).result().get_counts()

        #print(counts)

        probs = {}
        for output in counts.keys():
            
            probs[output] = counts[output]/shots



        print(probs)



        if measurement in probs.keys():
            Energy_meas.append(float(R[index_param].real) * (-sum(probs.values()) + 2 * probs['0000'] + 2 * probs[measurement]))
        else:
            Energy_meas.append(0)
        print(Energy_meas)

        E_sim.append(np.sum(np.array(Energy_meas)))

    return (E_sim, Energy_meas, R, counts, measurement, hamiltonian_circuit)
    
        #E_sim[0] + float(R[0].real) + float(R[num_elements_hamiltonian].real)

In [132]:
pi = np.pi

theta = np.array([[1, 0, 1, 2],
                  [1, 0, 1, 2],
                  [1, 0, 1, 2],
                  [1, 0, 1, 2]])

R = [24.674011 +0.j, -2.4674011+0.j, -9.8696044+0.j, -2.4674011+0.j, -9.8696044+0.j]

num_qubits = 4

gates_list_hamiltonian = ['0000', '0001', '0010', '0100', '1000']

num_elements_hamiltonian = 1

a = Energy_general(theta, R, num_qubits, gates_list_hamiltonian, num_elements_hamiltonian)

{'0100': 0.01214599609375, '0000': 0.0252685546875, '0001': 0.126220703125, '1000': 0.1038818359375, '1001': 0.49652099609375, '0101': 0.05877685546875, '1100': 0.02862548828125, '1101': 0.1485595703125}
[1.7198315772705077]
{'0100': 0.0126953125, '0000': 0.02484130859375, '1001': 0.50091552734375, '1000': 0.10247802734375, '0001': 0.1297607421875, '1100': 0.0299072265625, '0101': 0.06048583984375, '1101': 0.138916015625}
[1.7198315772705077, 0]
{'0100': 0.0120849609375, '1101': 0.14447021484375, '0001': 0.13214111328125, '1000': 0.102294921875, '1001': 0.4923095703125, '0101': 0.060302734375, '1100': 0.03094482421875, '0000': 0.02545166015625}
[1.7198315772705077, 0, 2.282165299645996]
{'1101': 0.14959716796875, '0001': 0.1258544921875, '1100': 0.02996826171875, '0101': 0.060546875, '1000': 0.10162353515625, '1001': 0.49249267578125, '0000': 0.0269775390625, '0100': 0.012939453125}
[1.7198315772705077, 0, 2.282165299645996, 7.331120944091797]


In [133]:
a

([1.7198315772705077,
  1.7198315772705077,
  4.001996876916504,
  11.333117821008301],
 [1.7198315772705077, 0, 2.282165299645996, 7.331120944091797],
 [(24.674011+0j),
  (-2.4674011+0j),
  (-9.8696044+0j),
  (-2.4674011+0j),
  (-9.8696044+0j)],
 {'1101': 2451,
  '0001': 2062,
  '1100': 491,
  '0101': 992,
  '1000': 1665,
  '1001': 8069,
  '0000': 442,
  '0100': 212},
 '1000',
 <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fd4b70b2350>)

In [135]:
1.7198315772705077 + float(R[0].real) + float(R[num_elements_hamiltonian].real)

23.926441477270508

In [ ]:
E_sim[0] + float(R[0].real) + float(R[num_elements_hamiltonian].real)

In [114]:
a[-1].draw()

q315_0: ────────────
                 ┌─┐
q315_1: ─────────┤M├
              ┌─┐└╥┘
q315_2: ──────┤M├─╫─
           ┌─┐└╥┘ ║ 
q315_3: ───┤M├─╫──╫─
        ┌─┐└╥┘ ║  ║ 
q315_4: ┤M├─╫──╫──╫─
        └╥┘ ║  ║  ║ 
c114: 5/═╩══╩══╩══╩═
         4  3  2  1